<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/docs/examples/node_postprocessor/JinaRerank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jina Rerank

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [ ]:
!pip install llama-index-postprocessor-jinaai-rerank
!pip install llama-index-embeddings-jinaai
!pip install llama-index

In [ ]:
import os
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
)

In [ ]:
from llama_index.embeddings.jinaai import JinaEmbedding

api_key = os.environ["JINA_API_KEY"]
jina_embeddings = JinaEmbedding(api_key=api_key)

# load documents
import requests

url = "https://niketeam-asset-download.nike.net/catalogs/2024/2024_Nike%20Kids_02_09_24.pdf?cb=09302022"
response = requests.get(url)
with open("Nike_Catalog.pdf", "wb") as f:
    f.write(response.content)
reader = SimpleDirectoryReader(input_files=["Nike_Catalog.pdf"])
documents = reader.load_data()

# build index
index = VectorStoreIndex.from_documents(
    documents=documents, embed_model=jina_embeddings
)

#### Retrieve top 10 most relevant nodes, without using a reranker

In [ ]:
query_engine = index.as_query_engine(similarity_top_k=10)
response = query_engine.query(
    "What is the best jersey by Nike in terms of fabric?",
)

In [ ]:
print(response.source_nodes[0].text, response.source_nodes[0].score)
print("\n")
print(response.source_nodes[1].text, response.source_nodes[1].score)

56
Sustainable MaterialsNIKE KIDS  SOCCER – GOALKEEPER
KIDS NIKE DRY LS US PARK IV GK JERSEY 
CJ6073 $42.00
SIZES:  XS, S, M, L, XL
FABRIC:  100% polyester.
OFFER DATE:  04/01/20
END DATE:  12/31/25
Goal keepers jersey with graphic print on sleeves and across upper back panel, mesh back for breathability, 
slim fit with soft hand feel, shoulder seams rolled forward for better graphic visibility, straight seam across 
back, mesh back for breathability – gameday graphic print inspired by retro campos gk design .  
Body width: 16.3", Body length: 22" (size medium).
010 Black/White/(White) 012 Wolf Grey/White/(Black) 702 Volt/White/(Black)
KIDS NIKE DRY PARK III SHORT 
BV6866 $20.00
SIZES:  XS, S, M, L, XL
FABRIC:  100% polyester.
OFFER DATE:  04/01/20
END DATE:  12/31/25
Dri-FIT angled side seam short (slim fit) with soft hand feel updated fit for better mobility/comfort .  
Hip width: 16.9", Inseam length: 7" (size medium).
010 Black/White/(White) 012 Wolf Grey/Black/(Black) 702 Volt/(Bl

#### Retrieve top 10 most relevant nodes, but then rerank using Jina Reranker

By employing a reranker model, the prompt can be given more relevant context. This will lead to a more accurate response by the LLM.

In [ ]:
import os
from llama_index.postprocessor.jinaai_rerank import JinaRerank

jina_rerank = JinaRerank(api_key=api_key, top_n=2)

In [ ]:
query_engine = index.as_query_engine(
    similarity_top_k=10, node_postprocessors=[jina_rerank]
)
response = query_engine.query(
    "What is the best jersey by Nike in terms of fabric?",
)

In [ ]:
print(response.source_nodes[0].text, response.source_nodes[0].score)
print("\n")
print(response.source_nodes[1].text, response.source_nodes[1].score)

NIKE KIDS  SOCCER – STOCK41Sustainable Materials
Sustainable Materials
KIDS DRI-FIT ADV VAPOR IV JERSEY US SS
DR0837 $77.00
SIZES:  XS, S, M, L, XL
FABRIC:  100% polyester.
OFFER DATE:  01/01/23
END DATE:  12/31/24
Step on to the field ready for fast-paced play in the Nike Dri-FIT ADV Vapor Jersey. Engineered for 
optimal breathability, its moisture-wicking design helps keep you dry and cool under match-day pressure. 
Lightweight fabric in a relaxed, easy fit combats cling so you can focus on being the first to the ball. Lower 
insets line up perfectly with design details on the Nike Dri-FIT ADV Vapor IV Shorts to create an on-field 
look worthy of pro-level play. 
010  Black/Black/Black/(White) 100  White/White/White/(Black)
419  College Navy/College Navy/Game Royal/(White) 480  Game Royal/Game Royal/College Navy/(White)
657  University Red/University Red/Bright Crimson/(White)
BACK VIEW
GRAPHIC KNIT DETAIL
KIDS NIKE DRI-FIT US SS STRIKE III JERSEY
DR0913  $50.00
SIZES:  XS, S, M, L, 